In [1]:
testing_company = "AMZN"
winsize = 20

In [2]:
import os
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

path = "./data/"+testing_company+".csv"
temp = pd.read_csv(path)
data = temp["Adj Close"]

In [3]:
today = data[winsize:].values
yesterday = data[winsize-1:-1].values
target = (today - yesterday)/yesterday
temp = np.copy(target)
target[(temp>=-0.015)&(temp<=0.015)]=1
target[temp>0.015] = 0
target[temp<-0.015] = 2
target.shape

In [4]:
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()

adj = {"min":min(data), "max":max(data)}
data = scalar.fit_transform(data.values.reshape(-1,1))

In [5]:
adj

{'min': 659.090027, 'max': 3531.449951}

In [6]:
data

array([[0.0041116 ],
       [0.        ],
       [0.00517344],
       ...,
       [0.97906255],
       [0.97770822],
       [0.9495328 ]])

In [7]:
def split_data(stock, winsize=20):
    data = []
    for idx in range(len(stock) - winsize):
        data.append(stock[idx:idx+winsize])
        
    data = np.array(data)
    
    testsize = int(np.round(0.2*data.shape[0]))
    trainsize = data.shape[0]-testsize
    
    x_train = data[:trainsize, :-1].astype("float32")
    y_train = target[:trainsize].astype("float32")
    
    x_test = data[trainsize:,:-1].astype("float32")
    y_test = target[trainsize:].astype("float32")
    
    return x_train, y_train, x_test, y_test

In [8]:
#HYPERPARAMETERS
epoch = 100
lr = 0.001

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

x_train, y_train, x_test, y_test = split_data(data, winsize)
x_train = torch.from_numpy(x_train)
y_train = torch.from_numpy(y_train).long()
x_test = torch.from_numpy(x_test)
y_test = torch.from_numpy(y_test)


In [10]:
y_train[:20]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 0, 1])

In [11]:
import torch.nn.functional as F
class GRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn) = self.gru(x, (h0.detach()))
        out = self.fc(out[:, -1, :])
        out = F.relu(out)
        out = self.fc2(out)
        out = F.relu(out)
        out = self.fc3(out)
        return torch.softmax(out, dim=1)

In [12]:
input_dim = 1
hidden_dim = 32
output_dim = 3
layers = 2

model = GRU(input_dim, hidden_dim, layers, output_dim)
# criterion = nn.MSELoss(reduction="mean")
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)


In [13]:
x_train.shape

torch.Size([990, 19, 1])

In [14]:
history = []
batch = 100

for i in range(epoch):
    x = np.arange(x_train.shape[0])
    np.random.shuffle(x)
    for j in range(0, x_train.shape[0],batch):
        idx = x[j:min(j+batch, x_train.shape[0])]
        pred = model(x_train[idx])
        loss = criterion(pred, y_train[idx])
        history.append(loss.item())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print("Epoch {}({}/{}): loss={}".format(i, j, x_train.shape[0], loss.item()))


Epoch 0(0/990): loss=1.1493719816207886
Epoch 0(100/990): loss=1.1469078063964844
Epoch 0(200/990): loss=1.1491600275039673
Epoch 0(300/990): loss=1.1459702253341675
Epoch 0(400/990): loss=1.1549975872039795
Epoch 0(500/990): loss=1.1597579717636108
Epoch 0(600/990): loss=1.1398767232894897
Epoch 0(700/990): loss=1.1410260200500488
Epoch 0(800/990): loss=1.143934726715088
Epoch 0(900/990): loss=1.1520867347717285
Epoch 1(0/990): loss=1.1289949417114258
Epoch 1(100/990): loss=1.1387966871261597
Epoch 1(200/990): loss=1.1599653959274292
Epoch 1(300/990): loss=1.138983964920044
Epoch 1(400/990): loss=1.1576850414276123
Epoch 1(500/990): loss=1.1441377401351929
Epoch 1(600/990): loss=1.1589010953903198
Epoch 1(700/990): loss=1.1462234258651733
Epoch 1(800/990): loss=1.1484671831130981
Epoch 1(900/990): loss=1.1428546905517578
Epoch 2(0/990): loss=1.1557230949401855
Epoch 2(100/990): loss=1.1412159204483032
Epoch 2(200/990): loss=1.1470136642456055
Epoch 2(300/990): loss=1.1468374729156494


Epoch 19(600/990): loss=1.0928235054016113
Epoch 19(700/990): loss=1.0940088033676147
Epoch 19(800/990): loss=1.0940204858779907
Epoch 19(900/990): loss=1.0968278646469116
Epoch 20(0/990): loss=1.1012709140777588
Epoch 20(100/990): loss=1.094441533088684
Epoch 20(200/990): loss=1.092323660850525
Epoch 20(300/990): loss=1.0912785530090332
Epoch 20(400/990): loss=1.0919373035430908
Epoch 20(500/990): loss=1.0870518684387207
Epoch 20(600/990): loss=1.0875239372253418
Epoch 20(700/990): loss=1.0877288579940796
Epoch 20(800/990): loss=1.084441065788269
Epoch 20(900/990): loss=1.090090036392212
Epoch 21(0/990): loss=1.094524621963501
Epoch 21(100/990): loss=1.090272068977356
Epoch 21(200/990): loss=1.0870211124420166
Epoch 21(300/990): loss=1.0892106294631958
Epoch 21(400/990): loss=1.0866225957870483
Epoch 21(500/990): loss=1.0869073867797852
Epoch 21(600/990): loss=1.0865554809570312
Epoch 21(700/990): loss=1.086510419845581
Epoch 21(800/990): loss=1.0874871015548706
Epoch 21(900/990): los

Epoch 39(600/990): loss=1.025895118713379
Epoch 39(700/990): loss=1.0350877046585083
Epoch 39(800/990): loss=1.036805272102356
Epoch 39(900/990): loss=1.0256391763687134
Epoch 40(0/990): loss=1.0367437601089478
Epoch 40(100/990): loss=1.0348578691482544
Epoch 40(200/990): loss=1.022684097290039
Epoch 40(300/990): loss=1.0204286575317383
Epoch 40(400/990): loss=1.0219837427139282
Epoch 40(500/990): loss=1.040490984916687
Epoch 40(600/990): loss=1.0173171758651733
Epoch 40(700/990): loss=1.0333598852157593
Epoch 40(800/990): loss=1.0272306203842163
Epoch 40(900/990): loss=1.0304408073425293
Epoch 41(0/990): loss=1.0167864561080933
Epoch 41(100/990): loss=1.0281649827957153
Epoch 41(200/990): loss=1.0188406705856323
Epoch 41(300/990): loss=1.0340803861618042
Epoch 41(400/990): loss=1.0368876457214355
Epoch 41(500/990): loss=1.0408364534378052
Epoch 41(600/990): loss=1.0070899724960327
Epoch 41(700/990): loss=1.022447109222412
Epoch 41(800/990): loss=1.0298734903335571
Epoch 41(900/990): l

Epoch 59(400/990): loss=0.9885948896408081
Epoch 59(500/990): loss=0.9627042412757874
Epoch 59(600/990): loss=0.9955990314483643
Epoch 59(700/990): loss=0.9711797833442688
Epoch 59(800/990): loss=0.9520378112792969
Epoch 59(900/990): loss=0.9760201573371887
Epoch 60(0/990): loss=0.9560136198997498
Epoch 60(100/990): loss=0.983921229839325
Epoch 60(200/990): loss=0.9577203392982483
Epoch 60(300/990): loss=0.985737144947052
Epoch 60(400/990): loss=0.9839754700660706
Epoch 60(500/990): loss=0.9768193960189819
Epoch 60(600/990): loss=0.9688827395439148
Epoch 60(700/990): loss=0.9741565585136414
Epoch 60(800/990): loss=0.958281934261322
Epoch 60(900/990): loss=0.9804432392120361
Epoch 61(0/990): loss=0.9515939354896545
Epoch 61(100/990): loss=0.999491810798645
Epoch 61(200/990): loss=0.9525529742240906
Epoch 61(300/990): loss=0.9575861096382141
Epoch 61(400/990): loss=0.962175726890564
Epoch 61(500/990): loss=0.984793484210968
Epoch 61(600/990): loss=0.971172571182251
Epoch 61(700/990): los

Epoch 79(400/990): loss=0.9125802516937256
Epoch 79(500/990): loss=0.9468927979469299
Epoch 79(600/990): loss=0.9449623823165894
Epoch 79(700/990): loss=0.910258412361145
Epoch 79(800/990): loss=0.9469425678253174
Epoch 79(900/990): loss=0.9221674799919128
Epoch 80(0/990): loss=0.9472593665122986
Epoch 80(100/990): loss=0.9355428218841553
Epoch 80(200/990): loss=0.9001117944717407
Epoch 80(300/990): loss=0.9884956479072571
Epoch 80(400/990): loss=0.9082489013671875
Epoch 80(500/990): loss=0.885667085647583
Epoch 80(600/990): loss=0.8853370547294617
Epoch 80(700/990): loss=0.975891649723053
Epoch 80(800/990): loss=0.9310322403907776
Epoch 80(900/990): loss=0.9374293684959412
Epoch 81(0/990): loss=0.88958340883255
Epoch 81(100/990): loss=0.9160699248313904
Epoch 81(200/990): loss=0.9469441175460815
Epoch 81(300/990): loss=0.9064064025878906
Epoch 81(400/990): loss=0.9703891277313232
Epoch 81(500/990): loss=0.930899441242218
Epoch 81(600/990): loss=0.9260193705558777
Epoch 81(700/990): lo

Epoch 99(300/990): loss=0.8699705004692078
Epoch 99(400/990): loss=0.9087001085281372
Epoch 99(500/990): loss=0.8975250124931335
Epoch 99(600/990): loss=0.9043205380439758
Epoch 99(700/990): loss=0.9206662178039551
Epoch 99(800/990): loss=0.8868386149406433
Epoch 99(900/990): loss=0.9555870294570923


In [15]:
#for testing
model.eval()
pred = model(x_test)

In [16]:
pred_class = pred.detach().numpy()
np.argmax(pred_class, axis=1)[-1]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1], dtype=int64)